In [5]:
import os
import re
import json
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import xml.etree.ElementTree as et
import io
import glob
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from tqdm import tqdm

#tqdm_notebook.pandas()

In [6]:
filePath = glob.glob("./new_data/combined_xml/*.xml")
col_names = ['nct_id', 'brief_title', 'official_title', 'overall_status', 
         'start_date','completion_date', 'phase', 'study_type', 
             'brief_summary', 'detailed_description', 'enrollment',
             'condition', 'intervention_name', 'eligibility']

name1 = ['nct_id','brief_title', 'official_title', 'overall_status', 
         'start_date','completion_date', 'phase', 'study_type']

name2 = ['brief_summary', 'detailed_description']

name3 = ['enrollment', 'condition']

df = pd.DataFrame(columns = col_names)
bad_file = []

for files in tqdm(filePath):
    try:
        with open(files, 'r', encoding="utf-8") as content:
            tree = et.parse(content)
            root = tree.getroot()

            row = {name: ''for name in col_names} 

            for d in tree.iter():
                for txt in d.iter():
                    if d.tag in name1:
                        row[d.tag] = d.text
                    if d.tag in name2:
                        row[d.tag] = d.getchildren()[0].text
                    if d.tag in name3:
                        row[d.tag] = txt.text
                    if d.tag == "intervention_name":
                        intSearch = root.find('intervention_browse')
                        if intSearch:
                            row[d.tag]= [i.text for i in intSearch.findall('mesh_term')]
                        else:
                            row[d.tag] = [i.text for i in root.iter('intervention_name')]
                    if d.tag =='eligibility':
                        row[d.tag] = root.find('eligibility').find('criteria').find('textblock').text
           
        df = df.append(pd.Series(row, index = col_names), ignore_index = True)
    except:
        bad_file.append(files)


100%|██████████| 61781/61781 [23:36<00:00, 43.62it/s]


In [1]:
print(len(df))
df['intervention_name'] = df['intervention_name'].apply(', '.join)
df.head()

NameError: name 'df' is not defined

In [7]:
df.to_csv('cancerTrials.csv', index=False)

In [8]:
bad_file

['./new_data/combined_xml\\NCT00039754.xml',
 './new_data/combined_xml\\NCT00076531.xml',
 './new_data/combined_xml\\NCT00275444.xml',
 './new_data/combined_xml\\NCT00442754.xml']

In [ ]:
# # Single file processing
# xml_file = "./new_data/combined_xml/NCT01473303.xml"
# tree = et.parse(xml_file)
# root = tree.getroot()
# col_names = ['nct_id', 'brief_title', 'official_title', 'overall_status', 
#          'start_date','completion_date', 'phase', 'study_type', 
#              'brief_summary', 'detailed_description', 
#              'intervention','condition', 'enrollment']

# name1 = ['nct_id','brief_title', 'official_title', 'overall_status', 
#          'start_date','completion_date', 'phase', 'study_type']

# name2 = ['brief_summary', 'detailed_description']
             
# name3 = ['intervention','condition', 'enrollment']

# df = pd.DataFrame(columns = col_names)
# row = {name: ''for name in col_names} 
# for d in tree.iter():
#     for txt in d.iter():
#         if d.tag in name1:
#             row[d.tag] = d.text
#         if d.tag in name2:
#             row[d.tag] = d.getchildren()[0].text
#         if d.tag in name3:
#             row[d.tag] = txt.text
# df = df.append(pd.Series(row, index = col_names), ignore_index = True)


In [ ]:
#df.tail()